Group recommendation system answering why not question related to atomic granularity, group granularity and position absenteeism

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandarallel
# Importing the required library for the task
import pandas as pd
import numpy as np
import scipy.stats
import itertools


from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# naming of required column for all data
columns_name = {'data' : ["user_id", "item_id", "rating", "timestamp"] ,
                'item' : ['movie_id', 'movie_title' , 'release_date' , 'video_release_date' ,
              'IMDb_URL' , 'unknown' , 'Action', 'Adventure' , 'Animation' ,
              'Children', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film_Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ],
              'user': ['user_id' , 'age' , 'gender' , 'occupation' , 'zip code'],
              'genre' : ['genre', 'genre_id'] 
              }

In [ ]:
#reading data from the file of google drive
data = pd.read_csv('/content/drive/MyDrive/recommenderassignment1/u.data', "\t", encoding="utf-8", names= columns_name['data'])
print(data.head(5))
print("---------------------------------------------")
print("Number of row of the given data is: ", len(data.user_id))
print("---------------------------------------------")
print(data.info())
print("---------------------------------------------")

   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596
---------------------------------------------
Number of row of the given data is:  100000
---------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   item_id    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB
None
---------------------------------------------


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# reading movies file from the drive
movies = pd.read_csv("/content/drive/MyDrive/recommenderassignment1/u.item", sep = '|' ,encoding="latin-1", names= columns_name['item'])
print('----------------------------------------------------')
print(movies.head(2))
print("----------------------------------------------------")
print("Total number of given movies is : ",len(movies))
print("----------------------------------------------------")
movies.drop(['video_release_date', 'release_date', 'IMDb_URL'], axis=1, inplace = True)
print(movies.head(2))
print("----------------------------------------------------")
movies=movies[0:20]

----------------------------------------------------
   movie_id       movie_title release_date  video_release_date  \
0         1  Toy Story (1995)  01-Jan-1995                 NaN   
1         2  GoldenEye (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   

   Adventure  Animation  Children  ...  Fantasy  Film_Noir  Horror  Musical  \
0          0          1         1  ...        0          0       0        0   
1          1          0         0  ...        0          0       0        0   

   Mystery  Romance  Sci-Fi  Thriller  War  Western  
0        0        0       0         0    0        0  
1        0        0       0         1    0        0  

[2 rows x 24 columns]
----------------------------------------------------
Total number of given movies is :  1682
------------

In [ ]:
#reading user file from the drive
user = pd.read_csv('/content/drive/MyDrive/recommenderassignment1/u.user', sep = "|", encoding="utf-8", names= columns_name['user'])
print(user.head(2))
print("----------------------------------------------------")
print(len(user.user_id))
print("----------------------------------------------------")
genres = pd.read_csv('/content/drive/MyDrive/recommenderassignment1/u.genre', "|", encoding="utf-8", names= columns_name['genre'])
print(genres.head(2))

   user_id  age gender  occupation zip code
0        1   24      M  technician    85711
1        2   53      F       other    94043
----------------------------------------------------
943
----------------------------------------------------
     genre  genre_id
0  unknown         0
1   Action         1


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
def get_movies_groupby_genre():
    genres = {k: [] for k in range(19)}
    for index, row in movies.iterrows():
        row_genre = np.array(row[3:])
        genres_index = np.argwhere(row_genre==1)
        for i in genres_index:
            genres[i[0]].append(row["movie_title"])
    return genres 

In [ ]:
def get_ratings_single_user(user_id):
    # get all movies rated by the user
    return data[data["user_id"] == user_id].sort_values(by=['item_id'])  

In [ ]:
def both_rated_set_for_group(group_ratings,user_ratings):

  group_ratings_similar = group_ratings[group_ratings["movie_id"].isin(user_ratings["item_id"])]
  user_ratings_similar  = user_ratings[user_ratings["item_id"].isin(group_ratings["movie_id"])]

  group_ratings_similar = group_ratings_similar.sort_values(by=['movie_id'])
  user_ratings_similar  = user_ratings_similar.sort_values(by=['item_id'])

  return group_ratings_similar,user_ratings_similar

In [ ]:
def correlation_pearson(usera,userb):
  #step 1get all ratings by 2 users
  usera_ratings = data[data['user_id'] == usera].sort_values(by=['item_id'])
  userb_ratings = data[data['user_id'] == userb].sort_values(by=['item_id'])
  #step 2 get movie id watched by both users dubai le hereko movie ko dataframe
  userab = usera_ratings[usera_ratings["item_id"].isin(userb_ratings["item_id"])].sort_values(by=['item_id'])
  userba = userb_ratings[userb_ratings["item_id"].isin(usera_ratings["item_id"])].sort_values(by=['item_id'])
  #mean
  usera_mean = np.mean(usera_ratings['rating'])
  userb_mean = np.mean(userb_ratings['rating'])
  #Subtract
  var_a = np.array(np.subtract(userab['rating'],[usera_mean]))
  var_b = np.array(np.subtract(userba['rating'],[userb_mean]))
  #correlation formula
  first =np.sum(var_a*var_b)
  second =np.sqrt(np.sum(np.power(var_a,2)))*np.sqrt(np.sum(np.power(var_b,2)))
  correlation = first/second
  return correlation,usera,userb

In [ ]:
def group_pearson_correlation(group_ratings,user_ratings):
  group_ratings_similar,user_ratings_similar = both_rated_set_for_group(group_ratings,user_ratings)
  mean_group_ratings = np.mean(group_ratings['average_aggregration'])
  mean_user_ratings = np.mean(user_ratings['rating'])
  var_1 = np.array(np.subtract(group_ratings_similar["average_aggregration"], [mean_group_ratings]))
  var_2 = np.array(np.subtract(user_ratings_similar["rating"], [mean_user_ratings]))
  numerator = np.sum(var_1*var_2)
  denominator = np.sqrt(np.sum(np.power(var_1, 2)))*np.sqrt(np.sum(np.power(var_2, 2)))
  correlation = numerator / denominator
  return correlation

In [ ]:
#top_ten users
def top_users(userid,num_of_users):
  test_users = user[user["user_id"] != userid]
  correlations_of_users = test_users.apply(lambda user:correlation_pearson(userid,user['user_id']), axis=1, result_type='expand')
  correlations_of_users.columns=['correlation','usera','userb']
  correlations_of_users = correlations_of_users.sort_values(by=['correlation'], ascending=False)
  similar_user = correlations_of_users.head(num_of_users)
  return similar_user

In [ ]:
#Calculate the item score for a user based on another user
def item_score_userab(usera,userb,item_id):
  #get user rating for an item
  usera_ratings = data[data['user_id'] == usera].sort_values(by=['item_id'])
  userb_ratings = data[data['user_id'] == userb].sort_values(by=['item_id'])
  #step 2 get movie id watched by both users dubai le hereko movie ko dataframe
  userba = userb_ratings[userb_ratings["item_id"].isin(usera_ratings["item_id"])].sort_values(by=['item_id'])
  #mean of user b
  userb_mean = np.mean(usera_ratings['rating'])
  #pearson correlation 
  correlation, usera ,userb =correlation_pearson(usera,userb)
  var_b = float(userb_ratings[userb_ratings["item_id"] == item_id]["rating"]) - userb_mean
  #output
  first = correlation *var_b
  second = correlation
  return [first,second]

In [ ]:
def user_movie_predict(user_id,item_id):
  #user ratings
   user_ratings = data[data['user_id'] == user_id].sort_values(by=['item_id'])
   #get mean user ratings
   user_ratings_mean= np.mean(user_ratings['rating'])
   #get other user who have rated the movie
   other_users = data[data['item_id'] ==item_id]
   #calculate item score
   score = other_users.apply(lambda user:item_score_userab(user_id,user['user_id'],item_id),axis=1, result_type='expand' )
   score= np.array(score)
   score = score[~np.isnan(score).any(axis=1), :]
   prediction_score = user_ratings_mean + np.sum(score[:,0]) / np.sum(score[:,1])

   return item_id, movies.at[item_id - 1, 'movie_title'], prediction_score

In [ ]:
def get_user_prediction(user_id):
  pandarallel.initialize(nb_workers=64,progress_bar=True)

  movies_score=movies.parallel_apply(lambda movie:user_movie_predict(user_id,movie['movie_id']),axis=1,result_type="expand")
  movies_score.columns = ['movie_id',"movie_title","prediction_rating"]
  return movies_score

In [ ]:
def agg_method(users,users_ratings):
  print(users_ratings,"user ratings")
  group_data=[users_ratings[0]["movie_id"],users_ratings[0]["movie_title"],users_ratings[0]["prediction_rating"],users_ratings[1]["prediction_rating"],users_ratings[2]["prediction_rating"]]
  headers = ["movie_id","movie_title", "user{} rating".format(str(users[0])), "user{}rating".format(str(users[1])), "user{}rating".format(str(users[2]))]
  users_ratings_combine_agg = pd.concat(group_data,axis=1,keys=headers)
  users_ratings_combine_agg['average_aggregration'] =  users_ratings_combine_agg.iloc[:,1:4].mean(axis=1)
  return users_ratings_combine_agg

In [ ]:
def group_recommender(users_list):
  users_ratings = []
  for user in users_list:
    user_ratings =get_user_prediction(user)
    users_ratings.append(user_ratings)
  group_ratings = agg_method(users_list,users_ratings)
  return group_ratings.sort_values(by=['average_aggregration'], ascending=False)

In [ ]:
demo = group_recommender([2,4,6])


INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second
<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second


INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second
<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second
<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second
<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second
<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second
<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second
<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first/second
<ipython-input-126-e988199713c1>:17: RuntimeWarning: invalid value encountered in double_scalars
  correlation = first

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[    movie_id                                        movie_title  \
0          1                                   Toy Story (1995)   
1          2                                   GoldenEye (1995)   
2          3                                  Four Rooms (1995)   
3          4                                  Get Shorty (1995)   
4          5                                     Copycat (1995)   
5          6  Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
6          7                              Twelve Monkeys (1995)   
7          8                                        Babe (1995)   
8          9                            Dead Man Walking (1995)   
9         10                                 Richard III (1995)   
10        11                               Seven (Se7en) (1995)   
11        12                         Usual Suspects, The (1995)   
12        13                            Mighty Aphrodite (1995)   
13        14                                 Postino, Il (199

<ipython-input-132-d186f6ba2b4c>:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  users_ratings_combine_agg['average_aggregration'] =  users_ratings_combine_agg.iloc[:,1:4].mean(axis=1)


In [ ]:
print(demo)

    movie_id                                        movie_title  user2 rating  \
5          6  Shanghai Triad (Yao a yao yao dao waipo qiao) ...      3.881714   
1          2                                   GoldenEye (1995)      3.090040   
7          8                                        Babe (1995)      3.947538   
13        14                                 Postino, Il (1994)      4.052797   
11        12                         Usual Suspects, The (1995)      4.446519   
18        19                              Antonia's Line (1995)      3.783061   
9         10                                 Richard III (1995)      3.207726   
0          1                                   Toy Story (1995)      3.900488   
12        13                            Mighty Aphrodite (1995)      3.678272   
6          7                              Twelve Monkeys (1995)      3.824165   
19        20                          Angels and Insects (1995)      3.672453   
14        15                

In [ ]:
def atomic_granularity_explanation(all_movies,movie_name,ratings,group_ratings,peers,closest_peers):
  reasons=[]
  if all_movies.loc[all_movies['movie_title'] == movie_name].shape[0] ==0:
    reasons.append("The movie is not in the database")
  movie_id = all_movies.loc[all_movies['movie_title'] == movie_name]['movie_id'].iloc[0]  
  movie_rating = group_ratings.loc[group_ratings['movie_id'] == movie_id]['average_aggregration'].iloc[0] # take avg rating of demo df movie id.
  same_rated_movies = group_ratings[group_ratings['average_aggregration'] == movie_rating] # movie rating barabar ko aru movie ko demo bata ko dataframe
  # determine similarly rated movies if it is in highest rank or not.
  if same_rated_movies.shape[0]:
    first_same_rated_movie_id = same_rated_movies.iloc[0]['movie_id']
    reindexed_group_ratings = group_ratings.reset_index(drop=True) # demo reindex from the highest avg rating.
    same_rated_movie_rating_index = reindexed_group_ratings.loc[reindexed_group_ratings['movie_id'] == first_same_rated_movie_id].index[0] # gives certain index value of movie id
    if same_rated_movie_rating_index <= 5:
      reasons.append("The predicted rating of this movie tallies with many other's in list")
      return reasons
  # Finding the ranking which is greater than index 5 at less than or equal to 10.
  movie_rating_index = reindexed_group_ratings.loc[reindexed_group_ratings['movie_id'] == movie_id].index[0]
  if 5 < movie_rating_index <= 10:
    reasons.append(f"This movie is ranked at position {movie_rating_index}. It will appear if you ask for more recommendations.")
    return reasons
 # Find peer ratings of this movie
  peer_ratings = ratings.loc[ratings['user_id'].isin(peers['user_id'])]
  movie_ratings = peer_ratings.loc[peer_ratings['item_id'] == movie_id]
  if movie_ratings.shape[0] == 0:
    reasons.append("This movie is not rated")
    return reasons

  if movie_ratings.shape[0] > 0:
    most_similar_peers_rated = movie_ratings.loc[movie_ratings['user_id'].isin(closest_peers["user_id"])]
    if most_similar_peers_rated.shape[0] < 5:
      reasons.append(f"Not enough similar peers rated this movie, only {most_similar_peers_rated.shape[0]} out of 5 did")
    if movie_ratings.shape[0] < 5:
      reasons.append(f"Not enough peers rated this movie, only {movie_ratings.shape[0]} out of 5 did")
  if len(reasons) > 0:
    return reasons
  return ["This movie should be in your recommended list"]

In [ ]:

  movie_id = demo.loc[demo['average_aggregration'] == 3.719567]
  # first_same_rated_movie_id = movie_id[0]['movie_id']

  print(movie_id.head(10))

Empty DataFrame
Columns: [movie_id, movie_title, user2 rating, user4rating, user6rating, average_aggregration]
Index: []


In [ ]:
def atomic_granularity_question(movie_name, users_ids, group_ratings):
  print(user)
  peers = user.loc[~user['user_id'].isin(users_ids)].sample(100)
  peers["similarity"] = peers.apply(lambda row: group_pearson_correlation(group_ratings,get_ratings_single_user(row["user_id"])), axis=1) # get similar users using pearson correlation
  closest_peers = peers.sort_values(by=['similarity'], ascending=False).head(10)
  reasons = atomic_granularity_explanation(movies, movie_name, data, group_ratings, peers, closest_peers)
  for reason in reasons:
    print(reason)


In [ ]:
peers = user.loc[~user['user_id'].isin([14, 4,6])].sample(100)
peers["similarity"] = peers.apply(lambda row: group_pearson_correlation(demo,get_ratings_single_user(row["user_id"])), axis=1)
print(peers.sort_values(by=['similarity'],ascending=False).head(10))

<ipython-input-127-786c46f387eb>:9: RuntimeWarning: invalid value encountered in double_scalars
  correlation = numerator / denominator


     user_id  age gender occupation zip code  similarity
930      931   60      M   educator    33556    1.000000
672      673   51      M   educator    22906    1.000000
661      662   55      M  librarian    19102    0.990225
479      480   57      M    retired    98257    0.940545
418      419   37      M     lawyer    43215    0.935043
565      566   20      M    student    14627    0.825044
638      639   42      F  librarian    12065    0.814188
709      710   19      M    student    92020    0.777127
784      785   32      M   engineer    23322    0.777127
668      669   37      M      other    20009    0.767550


In [ ]:
atomic_granularity_question("Mighty Aphrodite (1995)", [2, 4,6], demo)

     user_id  age gender     occupation zip code
0          1   24      M     technician    85711
1          2   53      F          other    94043
2          3   23      M         writer    32067
3          4   24      M     technician    43537
4          5   33      F          other    15213
..       ...  ...    ...            ...      ...
938      939   26      F        student    33319
939      940   32      M  administrator    02215
940      941   20      M        student    97229
941      942   48      F      librarian    78209
942      943   22      M        student    77841

[943 rows x 5 columns]


<ipython-input-127-786c46f387eb>:9: RuntimeWarning: invalid value encountered in double_scalars
  correlation = numerator / denominator


This movie is ranked at position 8. It will appear if you ask for more recommendations.


In [ ]:
def group_granularity_explanation(genre, all_genres, user_ids, movie_names, ratings, group_ratings, peers, closest_peers):
    genre_reasons = []
    reasons = []
    if all_genres.loc[all_genres['genre'] == genre].shape[0] == 0:
        genre_reasons.append("The genre is not in the database")
        return genre_reasons    
    for movie_name in movie_names:
        atomic_reasons = atomic_granularity_explanation(movies, movie_name, ratings, group_ratings, peers, closest_peers)
        reasons = reasons + atomic_reasons
    tie_reason = sum(1 if reason.startswith("This movie's predicted rating ties") else 0 for reason in reasons)
    low_rank_reason = sum(1 if reason.startswith("This movie is ranked at position") else 0 for reason in reasons)
    low_member_score_reason = sum(1 if reason.startswith("A member of the group rated this") else 0 for reason in reasons)
    not_rated_reason = sum(1 if reason.startswith("This movie is not rated") else 0 for reason in reasons)
    not_enough_similar_peers_reason = sum(1 if reason.startswith("Not enough similar peers rated this movie") else 0 for reason in reasons)
    not_enough_peers_reason = sum(1 if reason.startswith("Not enough peers rated this movie") else 0 for reason in reasons)
    movie_shoud_in_list = sum(1 if reason.startswith("This movie should be in your recommended list") else 0 for reason in reasons)
    
    main_reason = np.argmax([tie_reason, low_rank_reason, low_member_score_reason, not_rated_reason, not_enough_similar_peers_reason, not_enough_peers_reason, movie_shoud_in_list])
    
    main_reasons = {
        "0": "Too many tie ratings for movies in this genre",
        "1": f"Movies in this genre are in top {40}, show more to see the movies",
        "2": "Your group members rate movies in this genre low score.",
        "3": "Movies of this genre aren't rated much",
        "4": f"Your similar peers do not like {genre}",
        "5": f"Not enough peers rate {genre}",
        "6": "This genre should be in your recommended list"
    }
    
    return main_reasons[str(main_reason)]

In [ ]:
def group_granularity_question(genre, user_ids, group_ratings):    
    peers = user.loc[~user['user_id'].isin(user_ids)].sample(100)
    peers["similarity"] = peers.apply(lambda row: group_pearson_correlation(group_ratings, get_ratings_single_user(row["user_id"])), axis=1)
    closest_peers = peers.sort_values(by=['similarity'], ascending=False).head(10)
    movies_by_genre = get_movies_groupby_genre();    
    genre_id = genres.loc[genres["genre"] == "Adventure"]["genre_id"].iloc[0]
    all_movies = movies_by_genre[genre_id]
    reason = group_granularity_explanation(genre, genres, user_ids, all_movies, data, group_ratings, peers, closest_peers)
    print(reason)

In [ ]:
group_granularity_question("Adventure", [2, 17, 35], demo)

<ipython-input-127-786c46f387eb>:9: RuntimeWarning: invalid value encountered in double_scalars
  correlation = numerator / denominator


Movies in this genre are in top 40, show more to see the movies


In [ ]:
def position_absenteeism_explanation(all_movies, movie_name, ratings, group_ratings, peers, closest_peers, given_position=1):
    reasons = []
    if all_movies.loc[all_movies['movie_title'] == movie_name].shape[0] == 0:
        reasons.append("The movie is not in the database")
        return reasons
    movie_id = all_movies.loc[all_movies['movie_title'] == movie_name]['movie_id'].iloc[0]    
    movie_rating = group_ratings.loc[group_ratings['movie_id'] == movie_id]['average_aggregration'].iloc[0]
    same_rated_movies = group_ratings[group_ratings['average_aggregration'] == movie_rating]
    first_same_rated_movie_id = same_rated_movies.iloc[0]['movie_id']  # Find the highest ranking
    reindexed_group_ratings = group_ratings.reset_index(drop=True)
    same_rated_movie_rating_index = reindexed_group_ratings.loc[reindexed_group_ratings['movie_id'] == first_same_rated_movie_id].index[0]
    if same_rated_movies.shape[0] > 1 and same_rated_movie_rating_index <=20:
        reasons.append("This movie's predicted rating ties with many others")
        return reasons
    desired_position_movie_rating = reindexed_group_ratings['average_aggregration'].iloc[given_position-1] # Finding given position rating
    peer_ratings = ratings.loc[ratings['user_id'].isin(peers['user_id'])]  # Find peer ratings
    movie_ratings = peer_ratings.loc[peer_ratings['item_id'] == movie_id]
    movie_ratings_low_score = movie_ratings[movie_ratings['rating'] <= desired_position_movie_rating] # lower score
    most_similar_peers_rated = movie_ratings.loc[movie_ratings['user_id'].isin(closest_peers["user_id"])] # Find closest peers ratings
    most_similar_peers_rated_low_score = most_similar_peers_rated[most_similar_peers_rated['rating'] <= desired_position_movie_rating]
    if most_similar_peers_rated_low_score.shape[0] >= 25:
        reasons.append(f"{most_similar_peers_rated_low_score.shape[0]} of {closest_peers.shape[0]} most similar peers give this movie a lower score")
    if movie_ratings_low_score.shape[0] >= 25:
        reasons.append(f"{movie_ratings_low_score.shape[0]} of {peers.shape[0]} peers give this movie a lower score")
    if len(reasons) > 0:
        return reasons
    else:
        return [f"{peers.shape[0] - movie_ratings_low_score.shape[0]} of peers really like it, but {movie_ratings_low_score.shape[0]} do not enjoy it as much"]

In [ ]:
def position_absenteeism_question(movie_name, user_ids, group_ratings,given_position=1):    
    peers = user.loc[~user['user_id'].isin(user_ids)].sample(100)
    peers["similarity"] = peers.apply(lambda row: group_pearson_correlation(group_ratings, get_ratings_single_user(row["user_id"])), axis=1)
    closest_peers = peers.sort_values(by=['similarity'], ascending=False).head(10)
    min_bad_rated_similar_peers = 10/2
    reasons = position_absenteeism_explanation(movies, movie_name, data, group_ratings, peers, closest_peers, given_position)
    for reason in reasons:
        print(reason)

In [ ]:
position_absenteeism_question("Toy Story (1995)", [2, 17, 35], demo, 4)

<ipython-input-127-786c46f387eb>:9: RuntimeWarning: invalid value encountered in double_scalars
  correlation = numerator / denominator


35 of 100 peers give this movie a lower score
